<a href="https://colab.research.google.com/github/Worachet-Ch/project2022/blob/main/random_nan_and_imputation_with_BMI_variable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd #นำเข้า pandas แทนที่ด้วย pd
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as mno
import missingno as msno
from sklearn import linear_model
%matplotlib inline
import matplotlib
import sklearn 
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import time

In [2]:
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/project' #เข้า drive
data_file_path = os.path.join(path,'data_notmissing.csv') #ไปที่ไฟล์ confirmed-cases-since-280265.csv
data = pd.read_csv(data_file_path) #อ่านไฟล์ csv

Mounted at /content/drive


In [3]:
data.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)

<ipython-input-3-4cc622f169ae>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)


In [4]:
data

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [5]:
t_start_t = time.time()

# 10 % random nan

In [8]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'BMI10_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*10/100)) 
  index_nan = np.random.choice(globals()[f'BMI10_{i}']['BMI'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'BMI10_{i}']['BMI'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.17774605751037598 วินาที
เวลาที่ใช้ในการประมวลผล 0.0029624342918395997 นาที


## Imputation using Mean

In [9]:
from sklearn.impute import SimpleImputer

In [10]:
t_start = time.time()
mean10_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean10_{i}'] = mean10_imputer.fit_transform(globals()[f'BMI10_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN10_{i}'] = mean_squared_error(data, globals()[f'imputed_mean10_{i}']) # for loop mean_squared_error
t_end = time.time()
t10_MEAN= (t_end- t_start)

In [11]:
t_start = time.time()
mse_mean10_total = 0 
MSE_MEAN10_1001 = 0
for i in range(0,1001):
  mse_mean10_total = mse_mean10_total + globals()[f'MSE_MEAN10_{i+1}']
AMSE_BMI10_mean = mse_mean10_total/1000
print(f'AMSE_BMI10_mean = {AMSE_BMI10_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI10_mean = 0.01936803813539053
เวลาที่ใช้ในการประมวลผล 0.009600162506103516 วินาที
เวลาที่ใช้ในการประมวลผล 0.0001600027084350586 นาที


## Imputation using the miss forest

In [12]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 5.9 MB/s eta 0:00:00


In [ ]:
t_start = time.time()
MF10_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf10_{i}'] = MF10_imputer.fit_transform(globals()[f'BMI10_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF10_{i}'] = mean_squared_error(data, globals()[f'imputed_mf10_{i}']) # for loop mean_squared_error
t_end = time.time()
t10_MF= (t_end- t_start)

In [14]:
t_start = time.time()
mse_mf10_total = 0 
MSE_MF10_1001 = 0
for i in range(0,1001):
  mse_mf10_total = mse_mf10_total + globals()[f'MSE_MF10_{i+1}']
AMSE_BMI10_mf = mse_mf10_total/1000
print(f'AMSE_BMI10_mf = {AMSE_BMI10_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI10_mf = 0.004330174669683257
เวลาที่ใช้ในการประมวลผล 0.0024216175079345703 วินาที
เวลาที่ใช้ในการประมวลผล 4.0360291798909505e-05 นาที


## Imputation using the KNN

In [15]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [24]:
t_start = time.time()
KNN10_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN10_{i}'] = KNN10_imputer.fit_transform(globals()[f'BMI10_{i}'].copy())
  globals()[f'MSE_KNN10_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN10_{i}'])
t_end = time.time()
t10_KNN= (t_end- t_start)

In [25]:
t_start = time.time()
mse_knn10_total = 0 
MSE_KNN10_1001 = 0
for i in range(0,1001):
  mse_knn10_total = mse_knn10_total + globals()[f'MSE_KNN10_{i+1}']
AMSE_BMI10_knn = mse_knn10_total/1000
print(f'AMSE_BMI10_knn = {AMSE_BMI10_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI10_knn = 0.010301678811239588
เวลาที่ใช้ในการประมวลผล 0.001134634017944336 วินาที
เวลาที่ใช้ในการประมวลผล 1.891056696573893e-05 นาที


## Imputation using the MICE

In [26]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr10_{i}'] = imp.fit_transform(globals()[f'BMI10_{i}'].copy())
  globals()[f'MSE_lr10_{i}'] = mean_squared_error(data, globals()[f'imputed_lr10_{i}'])
t_end = time.time()
t10_LR= (t_end- t_start)

In [28]:
t_start = time.time()
mse_lr10_total = 0 
MSE_lr10_1001 = 0
for i in range(0,1001):
  mse_lr10_total = mse_lr10_total + globals()[f'MSE_lr10_{i+1}']
AMSE_BMI10_lr = mse_lr10_total/1000
print(f'AMSE_BMI10_lr = {AMSE_BMI10_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI10_lr = 0.0007325576111377501
เวลาที่ใช้ในการประมวลผล 0.0009014606475830078 วินาที
เวลาที่ใช้ในการประมวลผล 1.5024344126383463e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm10_{i}'] = svm_svr.fit_transform(globals()[f'BMI10_{i}'].copy())
  globals()[f'MSE_svm10_{i}'] = mean_squared_error(data, globals()[f'imputed_svm10_{i}'])
t_end = time.time()
t10_SVM = (t_end- t_start)

In [32]:
t_start = time.time()
mse_svm10_total = 0 
MSE_svm10_1001 = 0
for i in range(0,1001):
  mse_svm10_total = mse_svm10_total + globals()[f'MSE_svm10_{i+1}']
AMSE_BMI10_svm = mse_svm10_total/1000
print(f'AMSE_BMI10_svm = {AMSE_BMI10_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI10_svm = 0.010259359638894579
เวลาที่ใช้ในการประมวลผล 0.0011610984802246094 วินาที
เวลาที่ใช้ในการประมวลผล 1.9351641337076824e-05 นาที


## Imputation using the EM

### impy.em

In [33]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy10_{i}'] = impy.em(globals()[f'BMI10_{i}'].values.copy())
  globals()[f'MSE_impy10_{i}'] = mean_squared_error(data, globals()[f'imputed_impy10_{i}'])
t_end = time.time()
t10_EM = (t_end- t_start)

In [38]:
t_start = time.time()
mse_impy10_total = 0 
MSE_impy10_1001 = 0
for i in range(0,1001):
  mse_impy10_total = mse_impy10_total + globals()[f'MSE_impy10_{i+1}']
AMSE_BMI10_impy = mse_impy10_total/1000
print(f'AMSE_BMI10_impy = {AMSE_BMI10_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI10_impy = 0.03508214003430212
เวลาที่ใช้ในการประมวลผล 0.001237630844116211 วินาที
เวลาที่ใช้ในการประมวลผล 2.0627180735270182e-05 นาที


# 20 % random nan

In [39]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'BMI20_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*20/100)) 
  index_nan = np.random.choice(globals()[f'BMI20_{i}']['BMI'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'BMI20_{i}']['BMI'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.3857145309448242 วินาที
เวลาที่ใช้ในการประมวลผล 0.006428575515747071 นาที


In [40]:
BMI20_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [41]:
from sklearn.impute import SimpleImputer

In [42]:
t_start = time.time()
mean20_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean20_{i}'] = mean20_imputer.fit_transform(globals()[f'BMI20_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN20_{i}'] = mean_squared_error(data, globals()[f'imputed_mean20_{i}']) # for loop mean_squared_error
t_end = time.time()
t20_MEAN = (t_end- t_start)

In [43]:
t_start = time.time()
mse_mean20_total = 0 
MSE_MEAN20_1001 = 0
for i in range(0,1001):
  mse_mean20_total = mse_mean20_total + globals()[f'MSE_MEAN20_{i+1}']
AMSE_BMI20_mean = mse_mean20_total/1000
print(f'AMSE_BMI20_mean = {AMSE_BMI20_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI20_mean = 0.03822980408357405
เวลาที่ใช้ในการประมวลผล 0.0011830329895019531 วินาที
เวลาที่ใช้ในการประมวลผล 1.971721649169922e-05 นาที


## Imputation using the miss forest

In [44]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF20_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf20_{i}'] = MF20_imputer.fit_transform(globals()[f'BMI20_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF20_{i}'] = mean_squared_error(data, globals()[f'imputed_mf20_{i}']) # for loop mean_squared_error
t_end = time.time()
t20_MF = (t_end- t_start)

In [46]:
t_start = time.time()
mse_mf20_total = 0 
MSE_MF20_1001 = 0
for i in range(0,1001):
  mse_mf20_total = mse_mf20_total + globals()[f'MSE_MF20_{i+1}']
AMSE_BMI20_mf = mse_mf20_total/1000
print(f'AMSE_BMI20_mf = {AMSE_BMI20_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI20_mf = 0.008880307203871271
เวลาที่ใช้ในการประมวลผล 0.001550912857055664 วินาที
เวลาที่ใช้ในการประมวลผล 2.5848547617594402e-05 นาที


## Imputation using the KNN

In [47]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [48]:
t_start = time.time()
KNN20_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN20_{i}'] = KNN20_imputer.fit_transform(globals()[f'BMI20_{i}'].copy())
  globals()[f'MSE_KNN20_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN20_{i}'])
t_end = time.time()
t20_KNN = (t_end- t_start)

In [49]:
t_start = time.time()
mse_knn20_total = 0 
MSE_KNN20_1001 = 0
for i in range(0,1001):
  mse_knn20_total = mse_knn20_total + globals()[f'MSE_KNN20_{i+1}']
AMSE_BMI20_knn = mse_knn20_total/1000
print(f'AMSE_BMI20_knn = {AMSE_BMI20_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI20_knn = 0.020678804267918034
เวลาที่ใช้ในการประมวลผล 0.0008687973022460938 วินาที
เวลาที่ใช้ในการประมวลผล 1.4479955037434896e-05 นาที


## Imputation using the MICE

In [50]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr20_{i}'] = imp.fit_transform(globals()[f'BMI20_{i}'].copy())
  globals()[f'MSE_lr20_{i}'] = mean_squared_error(data, globals()[f'imputed_lr20_{i}'])
t_end = time.time()
t20_LR = (t_end- t_start)

In [52]:
t_start = time.time()
mse_lr20_total = 0 
MSE_lr20_1001 = 0
for i in range(0,1001):
  mse_lr20_total = mse_lr20_total + globals()[f'MSE_lr20_{i+1}']
AMSE_BMI20_lr = mse_lr20_total/1000
print(f'AMSE_BMI20_lr = {AMSE_BMI20_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI20_lr = 0.002034222396379551
เวลาที่ใช้ในการประมวลผล 0.0010402202606201172 วินาที
เวลาที่ใช้ในการประมวลผล 1.733700434366862e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm20_{i}'] = svm_svr.fit_transform(globals()[f'BMI20_{i}'].copy())
  globals()[f'MSE_svm20_{i}'] = mean_squared_error(data, globals()[f'imputed_svm20_{i}'])
t_end = time.time()
t20_SVM = (t_end- t_start)

In [54]:
t_start = time.time()
mse_svm20_total = 0 
MSE_svm20_1001 = 0
for i in range(0,1001):
  mse_svm20_total = mse_svm20_total + globals()[f'MSE_svm20_{i+1}']
AMSE_BMI20_svm = mse_svm20_total/1000
print(f'AMSE_BMI20_svm = {AMSE_BMI20_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI20_svm = 0.02119889562606676
เวลาที่ใช้ในการประมวลผล 0.0014717578887939453 วินาที
เวลาที่ใช้ในการประมวลผล 2.4529298146565757e-05 นาที


## Imputation using the EM

### impy.em

In [55]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy20_{i}'] = impy.em(globals()[f'BMI20_{i}'].values.copy())
  globals()[f'MSE_impy20_{i}'] = mean_squared_error(data, globals()[f'imputed_impy20_{i}'])
t_end = time.time()
t20_EM = (t_end- t_start)

In [57]:
t_start = time.time()
mse_impy20_total = 0 
MSE_impy20_1001 = 0
for i in range(0,1001):
  mse_impy20_total = mse_impy20_total + globals()[f'MSE_impy20_{i+1}']
AMSE_BMI20_impy = mse_impy20_total/1000
print(f'AMSE_BMI20_impy = {AMSE_BMI20_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI20_impy = 0.07180791774358229
เวลาที่ใช้ในการประมวลผล 0.002502918243408203 วินาที
เวลาที่ใช้ในการประมวลผล 4.1715304056803386e-05 นาที


# 30 % random nan

In [58]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'BMI30_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*30/100)) 
  index_nan = np.random.choice(globals()[f'BMI30_{i}']['BMI'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'BMI30_{i}']['BMI'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.19844627380371094 วินาที
เวลาที่ใช้ในการประมวลผล 0.0033074378967285155 นาที


In [59]:
BMI30_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [60]:
from sklearn.impute import SimpleImputer

In [61]:
t_start = time.time()
mean30_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean30_{i}'] = mean30_imputer.fit_transform(globals()[f'BMI30_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN30_{i}'] = mean_squared_error(data, globals()[f'imputed_mean30_{i}']) # for loop mean_squared_error
t_end = time.time()
t30_MEAN = (t_end- t_start)

In [62]:
t_start = time.time()
mse_mean30_total = 0 
MSE_MEAN30_1001 = 0
for i in range(0,1001):
  mse_mean30_total = mse_mean30_total + globals()[f'MSE_MEAN30_{i+1}']
AMSE_BMI30_mean = mse_mean30_total/1000
print(f'AMSE_BMI30_mean = {AMSE_BMI30_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI30_mean = 0.05790382071687489
เวลาที่ใช้ในการประมวลผล 0.001466512680053711 วินาที
เวลาที่ใช้ในการประมวลผล 2.4441878000895182e-05 นาที


## Imputation using the miss forest

In [63]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF30_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf30_{i}'] = MF30_imputer.fit_transform(globals()[f'BMI30_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF30_{i}'] = mean_squared_error(data, globals()[f'imputed_mf30_{i}']) # for loop mean_squared_error
t_end = time.time()
t30_MF = (t_end- t_start)

In [65]:
t_start = time.time()
mse_mf30_total = 0 
MSE_MF30_1001 = 0
for i in range(0,1001):
  mse_mf30_total = mse_mf30_total + globals()[f'MSE_MF30_{i+1}']
AMSE_BMI30_mf = mse_mf30_total/1000
print(f'AMSE_BMI30_mf = {AMSE_BMI30_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI30_mf = 0.01418671544544997
เวลาที่ใช้ในการประมวลผล 0.0010428428649902344 วินาที
เวลาที่ใช้ในการประมวลผล 1.7380714416503908e-05 นาที


## Imputation using the KNN

In [66]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [67]:
t_start = time.time()
KNN30_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN30_{i}'] = KNN30_imputer.fit_transform(globals()[f'BMI30_{i}'].copy())
  globals()[f'MSE_KNN30_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN30_{i}'])
t_end = time.time()
t30_KNN = (t_end- t_start)

In [68]:
t_start = time.time()
mse_knn30_total = 0 
MSE_KNN30_1001 = 0
for i in range(0,1001):
  mse_knn30_total = mse_knn30_total + globals()[f'MSE_KNN30_{i+1}']
AMSE_BMI30_knn = mse_knn30_total/1000
print(f'AMSE_BMI30_knn = {AMSE_BMI30_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI30_knn = 0.03153946617507399
เวลาที่ใช้ในการประมวลผล 0.0008554458618164062 วินาที
เวลาที่ใช้ในการประมวลผล 1.4257431030273437e-05 นาที


## Imputation using the MICE

In [69]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr30_{i}'] = imp.fit_transform(globals()[f'BMI30_{i}'].copy())
  globals()[f'MSE_lr30_{i}'] = mean_squared_error(data, globals()[f'imputed_lr30_{i}'])
t_end = time.time()
t30_LR = (t_end- t_start)

In [71]:
t_start = time.time()
mse_lr30_total = 0 
MSE_lr30_1001 = 0
for i in range(0,1001):
  mse_lr30_total = mse_lr30_total + globals()[f'MSE_lr30_{i+1}']
AMSE_BMI30_lr = mse_lr30_total/1000
print(f'AMSE_BMI30_lr = {AMSE_BMI30_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI30_lr = 0.0045367594133482216
เวลาที่ใช้ในการประมวลผล 0.0015501976013183594 วินาที
เวลาที่ใช้ในการประมวลผล 2.583662668863932e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm30_{i}'] = svm_svr.fit_transform(globals()[f'BMI30_{i}'].copy())
  globals()[f'MSE_svm30_{i}'] = mean_squared_error(data, globals()[f'imputed_svm30_{i}'])
t_end = time.time()
t30_SVM = (t_end- t_start)

In [73]:
t_start = time.time()
mse_svm30_total = 0 
MSE_svm30_1001 = 0
for i in range(0,1001):
  mse_svm30_total = mse_svm30_total + globals()[f'MSE_svm30_{i+1}']
AMSE_BMI30_svm = mse_svm30_total/1000
print(f'AMSE_BMI30_svm = {AMSE_BMI30_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI30_svm = 0.033512204174903165
เวลาที่ใช้ในการประมวลผล 0.0027587413787841797 วินาที
เวลาที่ใช้ในการประมวลผล 4.597902297973633e-05 นาที


## Imputation using the EM

### impy.em

In [74]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [75]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy30_{i}'] = impy.em(globals()[f'BMI30_{i}'].values.copy())
  globals()[f'MSE_impy30_{i}'] = mean_squared_error(data, globals()[f'imputed_impy30_{i}'])
t_end = time.time()
t30_EM = (t_end- t_start)

In [76]:
t_start = time.time()
mse_impy30_total = 0 
MSE_impy30_1001 = 0
for i in range(0,1001):
  mse_impy30_total = mse_impy30_total + globals()[f'MSE_impy30_{i+1}']
AMSE_BMI30_impy = mse_impy30_total/1000
print(f'AMSE_BMI30_impy = {AMSE_BMI30_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI30_impy = 0.10842059114812652
เวลาที่ใช้ในการประมวลผล 0.001741170883178711 วินาที
เวลาที่ใช้ในการประมวลผล 2.9019514719645183e-05 นาที


# 40 % random nan

In [77]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'BMI40_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*40/100)) 
  index_nan = np.random.choice(globals()[f'BMI40_{i}']['BMI'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'BMI40_{i}']['BMI'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.20461630821228027 วินาที
เวลาที่ใช้ในการประมวลผล 0.0034102718035380047 นาที


In [78]:
BMI40_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,NaN,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,NaN,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [79]:
from sklearn.impute import SimpleImputer

In [80]:
t_start = time.time()
mean40_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean40_{i}'] = mean40_imputer.fit_transform(globals()[f'BMI40_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN40_{i}'] = mean_squared_error(data, globals()[f'imputed_mean40_{i}']) # for loop mean_squared_error
t_end = time.time()
t40_MEAN = (t_end- t_start)

In [81]:
t_start = time.time()
mse_mean40_total = 0 
MSE_MEAN40_1001 = 0
for i in range(0,1001):
  mse_mean40_total = mse_mean40_total + globals()[f'MSE_MEAN40_{i+1}']
AMSE_BMI40_mean = mse_mean40_total/1000
print(f'AMSE_BMI40_mean = {AMSE_BMI40_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI40_mean = 0.07847464466783745
เวลาที่ใช้ในการประมวลผล 0.0013494491577148438 วินาที
เวลาที่ใช้ในการประมวลผล 2.2490819295247395e-05 นาที


## Imputation using the miss forest

In [82]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF40_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf40_{i}'] = MF40_imputer.fit_transform(globals()[f'BMI40_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF40_{i}'] = mean_squared_error(data, globals()[f'imputed_mf40_{i}']) # for loop mean_squared_error
t_end = time.time()
t40_MF = (t_end- t_start)

In [84]:
t_start = time.time()
mse_mf40_total = 0 
MSE_MF40_1001 = 0
for i in range(0,1001):
  mse_mf40_total = mse_mf40_total + globals()[f'MSE_MF40_{i+1}']
AMSE_BMI40_mf = mse_mf40_total/1000
print(f'AMSE_BMI40_mf = {AMSE_BMI40_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI40_mf = 0.020344384153594748
เวลาที่ใช้ในการประมวลผล 0.0011746883392333984 วินาที
เวลาที่ใช้ในการประมวลผล 1.9578138987223306e-05 นาที


## Imputation using the KNN

In [85]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [86]:
t_start = time.time()
KNN40_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN40_{i}'] = KNN40_imputer.fit_transform(globals()[f'BMI40_{i}'].copy())
  globals()[f'MSE_KNN40_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN40_{i}'])
t_end = time.time()
t40_KNN = (t_end- t_start)

In [87]:
t_start = time.time()
mse_knn40_total = 0 
MSE_KNN40_1001 = 0
for i in range(0,1001):
  mse_knn40_total = mse_knn40_total + globals()[f'MSE_KNN40_{i+1}']
AMSE_BMI40_knn = mse_knn40_total/1000
print(f'AMSE_BMI40_knn = {AMSE_BMI40_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI40_knn = 0.04347048821853518
เวลาที่ใช้ในการประมวลผล 0.0009124279022216797 วินาที
เวลาที่ใช้ในการประมวลผล 1.5207131703694661e-05 นาที


## Imputation using the MICE

In [88]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr40_{i}'] = imp.fit_transform(globals()[f'BMI40_{i}'].copy())
  globals()[f'MSE_lr40_{i}'] = mean_squared_error(data, globals()[f'imputed_lr40_{i}'])
t_end = time.time()
t40_LR = (t_end- t_start)

In [90]:
t_start = time.time()
mse_lr40_total = 0 
MSE_lr40_1001 = 0
for i in range(0,1001):
  mse_lr40_total = mse_lr40_total + globals()[f'MSE_lr40_{i+1}']
AMSE_BMI40_lr = mse_lr40_total/1000
print(f'AMSE_BMI40_lr = {AMSE_BMI40_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI40_lr = 0.006938295367362663
เวลาที่ใช้ในการประมวลผล 0.000988006591796875 วินาที
เวลาที่ใช้ในการประมวลผล 1.6466776529947916e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm40_{i}'] = svm_svr.fit_transform(globals()[f'BMI40_{i}'].copy())
  globals()[f'MSE_svm40_{i}'] = mean_squared_error(data, globals()[f'imputed_svm40_{i}'])
t_end = time.time()
t40_SVM = (t_end- t_start)

In [92]:
t_start = time.time()
mse_svm40_total = 0 
MSE_svm40_1001 = 0
for i in range(0,1001):
  mse_svm40_total = mse_svm40_total + globals()[f'MSE_svm40_{i+1}']
AMSE_BMI40_svm = mse_svm40_total/1000
print(f'AMSE_BMI40_svm = {AMSE_BMI40_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI40_svm = 0.04741023485610507
เวลาที่ใช้ในการประมวลผล 0.0013387203216552734 วินาที
เวลาที่ใช้ในการประมวลผล 2.2312005360921223e-05 นาที


## Imputation using the EM

### impy.em

In [93]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [94]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy40_{i}'] = impy.em(globals()[f'BMI40_{i}'].values.copy())
  globals()[f'MSE_impy40_{i}'] = mean_squared_error(data, globals()[f'imputed_impy40_{i}'])
t_end = time.time()
t40_EM = (t_end- t_start)

In [95]:
t_start = time.time()
mse_impy40_total = 0 
MSE_impy40_1001 = 0
for i in range(0,1001):
  mse_impy40_total = mse_impy40_total + globals()[f'MSE_impy40_{i+1}']
AMSE_BMI40_impy = mse_impy40_total/1000
print(f'AMSE_BMI40_impy = {AMSE_BMI40_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI40_impy = 0.14758660343289728
เวลาที่ใช้ในการประมวลผล 0.0009624958038330078 วินาที
เวลาที่ใช้ในการประมวลผล 1.604159673055013e-05 นาที


# 50 % random nan

In [96]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'BMI50_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*50/100)) 
  index_nan = np.random.choice(globals()[f'BMI50_{i}']['BMI'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'BMI50_{i}']['BMI'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.17898201942443848 วินาที
เวลาที่ใช้ในการประมวลผล 0.0029830336570739746 นาที


In [97]:
BMI50_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,NaN,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,NaN,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,NaN,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,NaN,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [98]:
from sklearn.impute import SimpleImputer

In [99]:
t_start = time.time()
mean50_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean50_{i}'] = mean50_imputer.fit_transform(globals()[f'BMI50_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN50_{i}'] = mean_squared_error(data, globals()[f'imputed_mean50_{i}']) # for loop mean_squared_error
t_end = time.time()
t50_MEAN = (t_end- t_start)

In [100]:
t_start = time.time()
mse_mean50_total = 0 
MSE_MEAN50_1001 = 0
for i in range(0,1001):
  mse_mean50_total = mse_mean50_total + globals()[f'MSE_MEAN50_{i+1}']
AMSE_BMI50_mean = mse_mean50_total/1000
print(f'AMSE_BMI50_mean = {AMSE_BMI50_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI50_mean = 0.0979904931478842
เวลาที่ใช้ในการประมวลผล 0.0014824867248535156 วินาที
เวลาที่ใช้ในการประมวลผล 2.470811208089193e-05 นาที


## Imputation using the miss forest

In [101]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF50_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf50_{i}'] = MF50_imputer.fit_transform(globals()[f'BMI50_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF50_{i}'] = mean_squared_error(data, globals()[f'imputed_mf50_{i}']) # for loop mean_squared_error
t_end = time.time()
t50_MF = (t_end- t_start)

In [103]:
t_start = time.time()
mse_mf50_total = 0 
MSE_MF50_1001 = 0
for i in range(0,1001):
  mse_mf50_total = mse_mf50_total + globals()[f'MSE_MF50_{i+1}']
AMSE_BMI50_mf = mse_mf50_total/1000
print(f'AMSE_BMI50_mf = {AMSE_BMI50_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI50_mf = 0.02690860875540468
เวลาที่ใช้ในการประมวลผล 0.001432657241821289 วินาที
เวลาที่ใช้ในการประมวลผล 2.3877620697021486e-05 นาที


## Imputation using the KNN

In [104]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [105]:
t_start = time.time()
KNN50_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN50_{i}'] = KNN50_imputer.fit_transform(globals()[f'BMI50_{i}'].copy())
  globals()[f'MSE_KNN50_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN50_{i}'])
t_end = time.time()
t50_KNN = (t_end- t_start)

In [106]:
t_start = time.time()
mse_knn50_total = 0 
MSE_KNN50_1001 = 0
for i in range(0,1001):
  mse_knn50_total = mse_knn50_total + globals()[f'MSE_KNN50_{i+1}']
AMSE_BMI50_knn = mse_knn50_total/1000
print(f'AMSE_BMI50_knn = {AMSE_BMI50_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI50_knn = 0.0561649139936316
เวลาที่ใช้ในการประมวลผล 0.0026586055755615234 วินาที
เวลาที่ใช้ในการประมวลผล 4.431009292602539e-05 นาที


## Imputation using the MICE

In [107]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr50_{i}'] = imp.fit_transform(globals()[f'BMI50_{i}'].copy())
  globals()[f'MSE_lr50_{i}'] = mean_squared_error(data, globals()[f'imputed_lr50_{i}'])
t_end = time.time()
t50_LR = (t_end- t_start)

In [109]:
t_start = time.time()
mse_lr50_total = 0 
MSE_lr50_1001 = 0
for i in range(0,1001):
  mse_lr50_total = mse_lr50_total + globals()[f'MSE_lr50_{i+1}']
AMSE_BMI50_lr = mse_lr50_total/1000
print(f'AMSE_BMI50_lr = {AMSE_BMI50_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI50_lr = 0.010596769895478584
เวลาที่ใช้ในการประมวลผล 0.0012433528900146484 วินาที
เวลาที่ใช้ในการประมวลผล 2.072254816691081e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm50_{i}'] = svm_svr.fit_transform(globals()[f'BMI50_{i}'].copy())
  globals()[f'MSE_svm50_{i}'] = mean_squared_error(data, globals()[f'imputed_svm50_{i}'])
t_end = time.time()
t50_SVM = (t_end- t_start)

In [111]:
t_start = time.time()
mse_svm50_total = 0 
MSE_svm50_1001 = 0
for i in range(0,1001):
  mse_svm50_total = mse_svm50_total + globals()[f'MSE_svm50_{i+1}']
AMSE_BMI50_svm = mse_svm50_total/1000
print(f'AMSE_BMI50_svm = {AMSE_BMI50_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI50_svm = 0.06212689812546626
เวลาที่ใช้ในการประมวลผล 0.0016186237335205078 วินาที
เวลาที่ใช้ในการประมวลผล 2.6977062225341798e-05 นาที


## Imputation using the EM

### impy.em

In [112]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [113]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy50_{i}'] = impy.em(globals()[f'BMI50_{i}'].values.copy())
  globals()[f'MSE_impy50_{i}'] = mean_squared_error(data, globals()[f'imputed_impy50_{i}'])
t_end = time.time()
t50_EM = (t_end- t_start)

In [114]:
t_start = time.time()
mse_impy50_total = 0 
MSE_impy50_1001 = 0
for i in range(0,1001):
  mse_impy50_total = mse_impy50_total + globals()[f'MSE_impy50_{i+1}']
AMSE_BMI50_impy = mse_impy50_total/1000
print(f'AMSE_BMI50_impy = {AMSE_BMI50_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI50_impy = 0.18533223240122534
เวลาที่ใช้ในการประมวลผล 0.0012433528900146484 วินาที
เวลาที่ใช้ในการประมวลผล 2.072254816691081e-05 นาที


# 60 % random nan

In [115]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'BMI60_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*60/100)) 
  index_nan = np.random.choice(globals()[f'BMI60_{i}']['BMI'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'BMI60_{i}']['BMI'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.15906405448913574 วินาที
เวลาที่ใช้ในการประมวลผล 0.002651067574818929 นาที


In [116]:
BMI60_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,NaN,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,NaN,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,NaN,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,NaN,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,NaN,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [117]:
from sklearn.impute import SimpleImputer

In [118]:
t_start = time.time()
mean60_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean60_{i}'] = mean50_imputer.fit_transform(globals()[f'BMI60_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN60_{i}'] = mean_squared_error(data, globals()[f'imputed_mean60_{i}']) # for loop mean_squared_error
t_end = time.time()
t60_MEAN = (t_end- t_start)

In [119]:
t_start = time.time()
mse_mean60_total = 0 
MSE_MEAN60_1001 = 0
for i in range(0,1001):
  mse_mean60_total = mse_mean60_total + globals()[f'MSE_MEAN60_{i+1}']
AMSE_BMI60_mean = mse_mean60_total/1000
print(f'AMSE_BMI60_mean = {AMSE_BMI60_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI60_mean = 0.11816958765732342
เวลาที่ใช้ในการประมวลผล 0.0013768672943115234 วินาที
เวลาที่ใช้ในการประมวลผล 2.294778823852539e-05 นาที


## Imputation using the miss forest

In [120]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF60_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf60_{i}'] = MF60_imputer.fit_transform(globals()[f'BMI60_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF60_{i}'] = mean_squared_error(data, globals()[f'imputed_mf60_{i}']) # for loop mean_squared_error
t_end = time.time()
t60_MF = (t_end- t_start)

In [122]:
t_start = time.time()
mse_mf60_total = 0 
MSE_MF60_1001 = 0
for i in range(0,1001):
  mse_mf60_total = mse_mf60_total + globals()[f'MSE_MF60_{i+1}']
AMSE_BMI60_mf = mse_mf60_total/1000
print(f'AMSE_BMI60_mf = {AMSE_BMI60_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI60_mf = 0.0347224759736048
เวลาที่ใช้ในการประมวลผล 0.0010716915130615234 วินาที
เวลาที่ใช้ในการประมวลผล 1.786152521769206e-05 นาที


## Imputation using the KNN

In [123]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [124]:
t_start = time.time()
KNN60_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN60_{i}'] = KNN60_imputer.fit_transform(globals()[f'BMI60_{i}'].copy())
  globals()[f'MSE_KNN60_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN60_{i}'])
t_end = time.time()
t60_KNN = (t_end- t_start)

In [125]:
t_start = time.time()
mse_knn60_total = 0 
MSE_KNN60_1001 = 0
for i in range(0,1001):
  mse_knn60_total = mse_knn60_total + globals()[f'MSE_KNN60_{i+1}']
AMSE_BMI60_knn = mse_knn60_total/1000
print(f'AMSE_BMI60_knn = {AMSE_BMI60_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI60_knn = 0.07168601056924193
เวลาที่ใช้ในการประมวลผล 0.003670930862426758 วินาที
เวลาที่ใช้ในการประมวลผล 6.118218104044597e-05 นาที


## Imputation using the MICE

In [126]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr60_{i}'] = imp.fit_transform(globals()[f'BMI60_{i}'].copy())
  globals()[f'MSE_lr60_{i}'] = mean_squared_error(data, globals()[f'imputed_lr60_{i}'])
t_end = time.time()
t60_LR = (t_end- t_start)

In [128]:
t_start = time.time()
mse_lr60_total = 0 
MSE_lr60_1001 = 0
for i in range(0,1001):
  mse_lr60_total = mse_lr60_total + globals()[f'MSE_lr60_{i+1}']
AMSE_BMI60_lr = mse_lr60_total/1000
print(f'AMSE_BMI60_lr = {AMSE_BMI60_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI60_lr = 0.01646251402491945
เวลาที่ใช้ในการประมวลผล 0.0009086132049560547 วินาที
เวลาที่ใช้ในการประมวลผล 1.5143553415934245e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm60_{i}'] = svm_svr.fit_transform(globals()[f'BMI60_{i}'].copy())
  globals()[f'MSE_svm60_{i}'] = mean_squared_error(data, globals()[f'imputed_svm60_{i}'])
t_end = time.time()
t60_SVM = (t_end- t_start)

In [130]:
t_start = time.time()
mse_svm60_total = 0 
MSE_svm60_1001 = 0
for i in range(0,1001):
  mse_svm60_total = mse_svm60_total + globals()[f'MSE_svm60_{i+1}']
AMSE_BMI60_svm = mse_svm60_total/1000
print(f'AMSE_BMI60_svm = {AMSE_BMI60_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI60_svm = 0.07886696439607331
เวลาที่ใช้ในการประมวลผล 0.0016393661499023438 วินาที
เวลาที่ใช้ในการประมวลผล 2.732276916503906e-05 นาที


## Imputation using the EM

### impy.em

In [131]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [132]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy60_{i}'] = impy.em(globals()[f'BMI60_{i}'].values.copy())
  globals()[f'MSE_impy60_{i}'] = mean_squared_error(data, globals()[f'imputed_impy60_{i}'])
t_end = time.time()
t60_EM = (t_end- t_start)

In [133]:
t_start = time.time()
mse_impy60_total = 0 
MSE_impy60_1001 = 0
for i in range(0,1001):
  mse_impy60_total = mse_impy60_total + globals()[f'MSE_impy60_{i+1}']
AMSE_BMI60_impy = mse_impy60_total/1000
print(f'AMSE_BMI60_impy = {AMSE_BMI60_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_BMI60_impy = 0.22400918157003302
เวลาที่ใช้ในการประมวลผล 0.000972747802734375 วินาที
เวลาที่ใช้ในการประมวลผล 1.621246337890625e-05 นาที


# AMSE

In [134]:
amse_value = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],
              '10%':[AMSE_BMI10_mean,AMSE_BMI10_mf,AMSE_BMI10_knn,AMSE_BMI10_lr,AMSE_BMI10_svm,AMSE_BMI10_impy],
      '20%':[AMSE_BMI20_mean,AMSE_BMI20_mf,AMSE_BMI20_knn,AMSE_BMI20_lr,AMSE_BMI20_svm,AMSE_BMI20_impy],
      '30%':[AMSE_BMI30_mean,AMSE_BMI30_mf,AMSE_BMI30_knn,AMSE_BMI30_lr,AMSE_BMI30_svm,AMSE_BMI30_impy],
      '40%':[AMSE_BMI40_mean,AMSE_BMI40_mf,AMSE_BMI40_knn,AMSE_BMI40_lr,AMSE_BMI40_svm,AMSE_BMI40_impy],
      '50%':[AMSE_BMI50_mean,AMSE_BMI50_mf,AMSE_BMI50_knn,AMSE_BMI50_lr,AMSE_BMI60_svm,AMSE_BMI50_impy]
      ,'60%':[AMSE_BMI60_mean,AMSE_BMI60_mf,AMSE_BMI60_knn,AMSE_BMI60_lr,AMSE_BMI60_svm,AMSE_BMI60_impy]}

In [135]:
data_AMSE = pd.DataFrame.from_dict(amse_value)
data_AMSE

,Method,10%,20%,30%,40%,50%,60%
0,MEAN,0.019368,0.038230,0.057904,0.078475,0.097990,0.118170
1,Miss Forest,0.004330,0.008880,0.014187,0.020344,0.026909,0.034722
2,KNN,0.010302,0.020679,0.031539,0.043470,0.056165,0.071686
3,MICE-LR,0.000733,0.002034,0.004537,0.006938,0.010597,0.016463
4,MICE-SVM,0.010259,0.021199,0.033512,0.047410,0.078867,0.078867
5,EM,0.035082,0.071808,0.108421,0.147587,0.185332,0.224009


# เวลาทั้งหมดในการรัน

### second

In [136]:
data_t_sec = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],'10%':[t10_MEAN,t10_MF,t10_KNN,t10_LR,t10_SVM,t10_EM],
      '20%':[t20_MEAN,t20_MF,t20_KNN,t20_LR,t20_SVM,t20_EM],'30%':[t30_MEAN,t30_MF,t30_KNN,t30_LR,t30_SVM,t30_EM],'40%':[t40_MEAN,t40_MF,t40_KNN,t40_LR,t40_SVM,t40_EM],
      '50%':[t50_MEAN,t50_MF,t50_KNN,t50_LR,t50_SVM,t50_EM],'60%':[t60_MEAN,t60_MF,t60_KNN,t60_LR,t60_SVM,t60_EM]}

In [137]:
data_second = pd.DataFrame.from_dict(data_t_sec)
data_second

,Method,10%,20%,30%,40%,50%,60%
0,MEAN,4.884439,3.778719,4.808225,4.874542,6.159041,3.828487
1,Miss Forest,721.266563,698.424743,675.580361,671.070441,671.142241,655.181990
2,KNN,5.464276,4.738069,4.864931,5.197400,8.305961,10.376151
3,MICE-LR,51.772371,51.275745,51.079265,52.910037,51.411433,52.068354
4,MICE-SVM,112.517722,112.658489,112.818627,112.553828,114.929476,114.268572
5,EM,6.367727,11.215493,17.444335,23.043270,28.215550,32.441371


### minutes

In [138]:
data_t_min = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],'10%':[t10_MEAN/60,t10_MF/60,t10_KNN/60,t10_LR/60,t10_SVM/60,t10_EM/60],
      '20%':[t20_MEAN/60,t20_MF/60,t20_KNN/60,t20_LR/60,t20_SVM/60,t20_EM/60],'30%':[t30_MEAN/60,t30_MF/60,t30_KNN/60,t30_LR/60,t30_SVM/60,t30_EM/60]
      ,'40%':[t40_MEAN/60,t40_MF/60,t40_KNN/60,t40_LR/60,t40_SVM/60,t40_EM/60],'50%':[t50_MEAN/60,t50_MF/60,t50_KNN/60,t50_LR/60,t50_SVM/60,t50_EM/60],
      '60%':[t60_MEAN/60,t60_MF/60,t60_KNN/60,t60_LR/60,t60_SVM/60,t60_EM/60]}

In [139]:
data_minutes = pd.DataFrame.from_dict(data_t_min)
data_minutes
     

,Method,10%,20%,30%,40%,50%,60%
0,MEAN,0.081407,0.062979,0.080137,0.081242,0.102651,0.063808
1,Miss Forest,12.021109,11.640412,11.259673,11.184507,11.185704,10.919700
2,KNN,0.091071,0.078968,0.081082,0.086623,0.138433,0.172936
3,MICE-LR,0.862873,0.854596,0.851321,0.881834,0.856857,0.867806
4,MICE-SVM,1.875295,1.877641,1.880310,1.875897,1.915491,1.904476
5,EM,0.106129,0.186925,0.290739,0.384054,0.470259,0.540690
